In [ ]:
!rm gen_image.tar.gz
!wget http://mireu-server.iptime.org/api/public/dl/QLwEhRq3/DataSet/%EA%B0%9D%EC%B2%B4%EC%9D%B8%EC%8B%9D_%EC%97%B0%EA%B5%AC/13.%ED%95%9C%EA%B5%AD%EC%96%B4%EA%B8%80%EC%9E%90%EC%B2%B4/image_gen/new_image/gen_image.tar.gz
!tar -xzvf gen_image.tar.gz
!pip3 install tqdm

In [ ]:
from PIL import Image
import numpy as np
import os

layerList = ["가", "나", "다"]
# , "라", "마", "바", "아", "자", "차", "카", "타", "파", "하"

all_x_array = []
all_y_array = []

for dir in layerList:
    fileList = os.listdir(f"../../dataset_dir/gen_image/{dir}")[:6000]

    for file in fileList:
        y_array = dict.fromkeys(layerList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성

        imageData = Image.open(f"""../../dataset_dir/gen_image/{dir}/{file}""") # 이미지 tensor 데이터 출력
        imageData = imageData.resize((32, 32)) # 이미지 다운스케일링

        imageData = np.array(imageData)
        all_x_array.append(imageData.tolist())

        y_array[dir] = 1 # 사용자가 지정한 str 값을 1로 변경
        y_array = list(y_array.values())
        all_y_array.append(y_array)


In [ ]:
x_array = np.array(all_x_array)
y_array = np.array(all_y_array)

In [ ]:
print(x_array.shape, y_array.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    x_array, y_array, test_size=0.2, shuffle=False, random_state=1004
)

In [ ]:
X_train = X_train.astype("float32") / 255
Y_train = Y_train.astype("float32") / 255

X_test = X_test.astype("float32") / 255
Y_test = Y_test.astype("float32") / 255

In [ ]:
image_w, image_h, image_layer = X_train.shape[1], X_train.shape[2], X_train.shape[3]

X_train = X_train.reshape(-1, image_w, image_h, image_layer)
X_test = X_test.reshape(-1, image_w, image_h, image_layer)

In [ ]:
DataSet_classes = Y_train.shape[1]

In [ ]:
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

In [ ]:
import tensorflow as tf

# tf.debugging.set_log_device_placement(True)

gpus = tf.config.list_physical_devices('GPU')
if gpus: # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')

  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

In [ ]:
import tensorflow as tf
import keras

model = keras.applications.VGG16(
    include_top=True,
    weights=None,
    input_shape=(image_w, image_h, image_layer),
    pooling=None,
    classes=DataSet_classes,
    classifier_activation="softmax",
)


In [ ]:
model.summary()

In [ ]:
batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(0.05), metrics=["accuracy"])

history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["loss"], label="loss") 
plt.plot(history.history["accuracy"], label="accuracy") 
plt.legend()
plt.show()